# Create ARC Awards

Creates awards from the Australian Research Council (ARC) grants data.

**Prerequisites:**
- Run `scripts/local/arc_to_s3.py` to download and upload the data first.

**Data source:** https://dataportal.arc.gov.au/RGS/Web/Grants  
**API:** https://dataportal.arc.gov.au/RGS/API/grants  
**S3 location:** `s3a://openalex-ingest/awards/arc/arc_projects.parquet`

**ARC funder:**
- funder_id: 4320334704
- display_name: "Australian Research Council"
- ROR: https://ror.org/05mmh0f86
- DOI: 10.13039/501100000923

**Notes:**
- ARC grants cover Discovery, Linkage, and National Intelligence programs
- Lead investigator name is in string format (e.g., "Prof John Smith")
- Currency is Australian Dollars (AUD)
- Grant schemes include Discovery Projects, DECRA, ARC Future Fellowships, Linkage Projects, etc.

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.arc_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/arc/arc_projects.parquet`;

In [ ]:
%sql
-- Check row count
SELECT COUNT(*) as total_grants FROM openalex.awards.arc_raw;

In [ ]:
%sql
-- Check column names and sample data
SELECT * FROM openalex.awards.arc_raw LIMIT 5;

In [ ]:
%sql
-- Check available columns
DESCRIBE openalex.awards.arc_raw;

## Step 2: Create ARC Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.arc_awards
USING delta
AS
WITH
-- Get ARC funder from OpenAlex by explicit funder_id
arc_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320334704  -- Australian Research Council
),

-- Parse lead investigator name into given/family names
-- ARC format is typically "Prof John Smith" or "Dr Jane Doe"
pi_parsed AS (
    SELECT
        *,
        -- Remove common titles and extract given name (first word after title)
        CASE
            WHEN lead_investigator IS NULL OR TRIM(lead_investigator) = '' THEN NULL
            WHEN lead_investigator LIKE '%,%' THEN 
                -- Format: "Family, Given" - take part after comma
                TRIM(SUBSTRING_INDEX(lead_investigator, ',', -1))
            ELSE
                -- Format: "Title Given Family" - extract given name
                TRIM(SPLIT(
                    TRIM(REGEXP_REPLACE(lead_investigator, '^(Dr|Prof|Professor|Associate Professor|A/Prof|Asst Prof|Assoc Prof|Mr|Ms|Mrs|Miss|Emeritus Professor|Emeritus Prof|Adj/Prof|Adjunct Professor)\\s+', '')),
                    ' '
                )[0])
        END as pi_given_name,
        CASE
            WHEN lead_investigator IS NULL OR TRIM(lead_investigator) = '' THEN NULL
            WHEN lead_investigator LIKE '%,%' THEN 
                -- Format: "Family, Given" - take part before comma
                TRIM(SUBSTRING_INDEX(lead_investigator, ',', 1))
            ELSE
                -- Format: "Title Given Family" - extract family name (last word)
                TRIM(ELEMENT_AT(
                    SPLIT(
                        TRIM(REGEXP_REPLACE(lead_investigator, '^(Dr|Prof|Professor|Associate Professor|A/Prof|Asst Prof|Assoc Prof|Mr|Ms|Mrs|Miss|Emeritus Professor|Emeritus Prof|Adj/Prof|Adjunct Professor)\\s+', '')),
                        ' '
                    ),
                    -1
                ))
        END as pi_family_name
    FROM openalex.awards.arc_raw
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:code
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.code)))) % 9000000000 as id,

        -- Display name = grant title
        g.title as display_name,

        -- Description from summary field
        g.summary as description,

        -- Funder info
        f.funder_id,
        g.code as funder_award_id,

        -- Amount in AUD - use TRY_CAST for safety
        TRY_CAST(g.grant_value AS DOUBLE) as amount,
        'AUD' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map ARC scheme types
        CASE
            WHEN LOWER(g.scheme_name) LIKE '%fellowship%' THEN 'fellowship'
            WHEN LOWER(g.scheme_name) LIKE '%decra%' OR LOWER(g.scheme_name) LIKE '%early career%' THEN 'fellowship'
            WHEN LOWER(g.scheme_name) LIKE '%laureate%' THEN 'fellowship'
            WHEN LOWER(g.scheme_name) LIKE '%infrastructure%' THEN 'infrastructure'
            WHEN LOWER(g.scheme_name) LIKE '%equipment%' THEN 'infrastructure'
            WHEN LOWER(g.scheme_name) LIKE '%facilities%' THEN 'infrastructure'
            WHEN LOWER(g.scheme_name) LIKE '%centre%' THEN 'infrastructure'
            WHEN LOWER(g.scheme_name) LIKE '%project%' THEN 'research'
            WHEN LOWER(g.scheme_name) LIKE '%discovery%' THEN 'research'
            WHEN LOWER(g.scheme_name) LIKE '%linkage%' THEN 'research'
            WHEN LOWER(g.scheme_name) LIKE '%industrial%' THEN 'research'
            WHEN LOWER(g.scheme_name) LIKE '%intelligence%' THEN 'research'
            WHEN LOWER(g.scheme_name) LIKE '%indigenous%' THEN 'research'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = scheme_name
        g.scheme_name as funder_scheme,

        -- Provenance
        'arc' as provenance,

        -- Dates - start_year is provided, end_date is provided as YYYY-MM-DD string
        CASE 
            WHEN TRY_CAST(g.start_year AS INT) IS NOT NULL 
            THEN TO_DATE(CONCAT(g.start_year, '-01-01')) 
            ELSE NULL 
        END as start_date,
        TRY_TO_DATE(g.end_date, 'yyyy-MM-dd') as end_date,
        TRY_CAST(g.start_year AS INT) as start_year,
        YEAR(TRY_TO_DATE(g.end_date, 'yyyy-MM-dd')) as end_year,

        -- Lead investigator
        CASE
            WHEN g.lead_investigator IS NOT NULL AND TRIM(g.lead_investigator) != '' THEN
                struct(
                    g.pi_given_name as given_name,
                    g.pi_family_name as family_name,
                    CAST(NULL AS STRING) as orcid,
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        g.grantee as name,
                        'Australia' as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not easily parseable from investigators string)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL - construct from code
        CONCAT('https://dataportal.arc.gov.au/RGS/Web/Grants/', g.code) as landing_page_url,

        -- No DOI for ARC grants
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.code)))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM pi_parsed g
    CROSS JOIN arc_funder f
    WHERE g.code IS NOT NULL
      AND TRIM(g.code) != ''
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Remove previous data for this source before inserting fresh data
DELETE FROM openalex.awards.openalex_awards_raw
WHERE provenance = 'arc' AND priority = 20;

-- Insert into openalex_awards_raw with priority
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    20 as priority
FROM openalex.awards.arc_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count
SELECT COUNT(*) as total_arc_awards FROM openalex.awards.arc_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    currency,
    start_date,
    end_date,
    lead_investigator.given_name as pi_given,
    lead_investigator.family_name as pi_family,
    lead_investigator.affiliation.name as institution,
    landing_page_url
FROM openalex.awards.arc_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be ARC)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.arc_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.arc_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution
SELECT funder_scheme, COUNT(*) as cnt, 
       ROUND(SUM(amount)/1e9, 2) as total_amount_billions_aud
FROM openalex.awards.arc_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_description,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(end_date) as has_end_date,
    COUNT(lead_investigator) as has_pi,
    ROUND(COUNT(display_name) * 100.0 / COUNT(*), 1) as pct_title,
    ROUND(COUNT(description) * 100.0 / COUNT(*), 1) as pct_description,
    ROUND(COUNT(amount) * 100.0 / COUNT(*), 1) as pct_amount,
    ROUND(COUNT(lead_investigator) * 100.0 / COUNT(*), 1) as pct_pi,
    ROUND(SUM(amount)/1e9, 2) as total_amount_billions_aud
FROM openalex.awards.arc_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt,
       ROUND(SUM(amount)/1e9, 2) as total_amount_billions_aud
FROM openalex.awards.arc_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 25;

In [ ]:
%sql
-- Top administering institutions
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as grant_count,
    ROUND(SUM(amount)/1e6, 1) as total_amount_millions_aud
FROM openalex.awards.arc_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY total_amount_millions_aud DESC
LIMIT 25;

In [ ]:
%sql
-- Check PI name parsing worked correctly
SELECT 
    lead_investigator.given_name as given_name,
    lead_investigator.family_name as family_name,
    COUNT(*) as cnt
FROM openalex.awards.arc_awards
WHERE lead_investigator IS NOT NULL
GROUP BY lead_investigator.given_name, lead_investigator.family_name
ORDER BY cnt DESC
LIMIT 20;